# Raw staking events to staker activity table Script
This script converts the staking events data returned by [Threshold Network Staking Event Dune query](https://dune.com/queries/4418882/7401185/) to a table that contains stake activity history.


In [ ]:
import pandas as pd

staking_events = pd.read_csv('StakingEvents2024-12-18.csv')

timestamp_list = staking_events['timestamp']
st_prov_list = staking_events.staking_provider.unique().tolist()

stakes_history = pd.DataFrame(
    index=timestamp_list, columns=st_prov_list, dtype=int)

for index, row in staking_events.iterrows():
    if index == 0:
        stakes_history.iloc[index] = 0
        stakes_history.at[row.timestamp, row.staking_provider] = int(row.amount)
    else:
        stakes_history.iloc[index] = stakes_history.iloc[index - 1]
        if row.event == "staked":
            stakes_history.at[row.timestamp, row.staking_provider] = int(row.amount)
        elif row.event == "toppedUp":
            stakes_history.at[row.timestamp,
                              row.staking_provider] += int(row.amount)
        elif row.event == "unstaked":
            stakes_history.at[row.timestamp,
                              row.staking_provider] -= int(row.amount)
        if (row.staking_provider == "0xcef1b4bf8f05f623a2a688b56d9da679d302eba7"):
            print(row.event, row.amount, int(row.amount), stakes_history.at[row.timestamp, row.staking_provider])

stakes_history.to_csv('StakesHistory2024-12-18.csv')

staked 2141060789870473260499710 2141060789870473260499710 2.1410607898704733e+24
unstaked 2141058997287102022089960 2141058997287102022089960 1.7925833712226796e+18
unstaked 1792583371238409750 1792583371238409750 -15730176.0
